pip install fastapi uvicorn pymysql pandas scikit-learn
pip install "fastapi[standard]"
pip install 
pip install rapidfuzz
WARNING: The script uvicorn.exe is installed in 'C:\Users\AdministraorK\AppData\Roaming\Python\Python313\Scripts' which is not on PATH.
 == 환경변수 편집집
 ### ctrl + shift + p 로 열어서 Python: Select Interpreter 선택 해서 python 환경 맞는거 선택
 


In [1]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import pymysql
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rapidfuzz import process


In [2]:
# ✅ FastAPI 인스턴스 생성
app = FastAPI()

In [3]:
# ✅ MariaDB 연결 정보
DB_HOST = "localhost"
DB_USER = "root"
DB_PASSWORD = "11111111"
DB_NAME = "navi_go"

In [4]:
# ✅ MariaDB 연결 함수
def get_connection():
    return pymysql.connect(
        host=DB_HOST,
        user=DB_USER,
        password=DB_PASSWORD,
        database=DB_NAME,
        charset="utf8mb4",
        cursorclass=pymysql.cursors.DictCursor
    )

In [5]:
# ✅ 사용자 선호도 가져오기 (preference 테이블에서 조회)
def get_user_preference(member_id):
    connection = get_connection()
    cursor = connection.cursor()
    sql = "SELECT prefer_purpose FROM preference WHERE member_id = %s"
    cursor.execute(sql, (member_id,))
    result = cursor.fetchone()  # 하나의 값만 가져옴
    connection.close()
    return result["prefer_purpose"] if result else None

In [6]:
# ✅ 사용자의 클릭 데이터 가져오기
def get_user_click_history(member_id):
    connection = get_connection()
    cursor = connection.cursor()
    sql = "SELECT contentid, cat1, cat2, cat3 FROM user_activity WHERE member_id = %s"
    cursor.execute(sql, (member_id,))
    result = cursor.fetchall()  # 모든 데이터(contentid, cat1, cat2, cat3)를 리스트로 반환
    connection.close()
    return result  # ✅ 리스트 형태로 반환

In [7]:
def load_category_data():
    file_path = "data/한국관광공사_국문_서비스분류코드_v4.2.xlsx"
    xls = pd.ExcelFile(file_path)
    df = pd.read_excel(xls, sheet_name='국문')

    df = df.iloc[4:]  # 필요없는 행 제거
    df.columns = ["contenttypeid", "cat1", "cat2", "cat3", "대분류", "중분류", "소분류"]
    df = df.dropna()
    return df  # ✅ DataFrame 반환

In [8]:
# ✅ 1. 최적화된 preference → cat2 매핑 (유사도 적용)

def map_preference_to_cat2(preference, df):
    choices = df["중분류"].tolist()  # 비교 대상 리스트
    best_match, score, index = process.extractOne(preference, choices)  # 가장 유사한 단어 찾기
    
    if score > 80:  # 유사도가 80 이상이면 매칭 (조절 가능)
        return df.iloc[index]["cat2"]  # 해당 row의 cat2 반환
    return None  # 매칭 실패 시 None 반환

In [9]:
# ✅ 2. AI 추천 모델 (유사도 기반 cat3 추천)
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def recommend_best_cat3(member_id):
    user_clicks = get_user_click_history(member_id)  # 사용자가 클릭한 여행지 정보 가져오기
    user_preference = get_user_preference(member_id)  # 사용자의 선호도
    category_data = load_category_data()  # Excel 카테고리 데이터 로드

    # ✅ 사용자의 선호도 → cat2 코드 변환 (유사도 매칭)
    preference_cat2 = map_preference_to_cat2(user_preference, category_data)

    # ✅ 클릭한 데이터가 없을 경우
    if not user_clicks:
        if preference_cat2:
            # 사용자의 preference(cat2)와 일치하는 cat3 중에서 인기 있는 것 추천
            best_cat3 = category_data[category_data["cat2"] == preference_cat2].head(3)
            return best_cat3[["cat3", "대분류", "중분류"]].to_dict(orient="records")
        return []  # 클릭 데이터도 없고, preference도 없으면 추천 안 함

    # ✅ 사용자가 클릭한 cat1, cat2, cat3 정보를 문자열로 결합하여 벡터화
    clicked_features = [" ".join([click["cat1"], click["cat2"], click["cat3"]]) for click in user_clicks]

    # ✅ Excel 데이터의 `cat1`, `cat2`, `cat3`를 벡터화
    category_data["features"] = category_data["cat1"] + " " + category_data["cat2"] + " " + category_data["cat3"]

    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(category_data["features"])
    clicked_vector = vectorizer.transform(clicked_features)

    # ✅ 클릭한 데이터와 유사한 여행지 찾기
    similarities = cosine_similarity(tfidf_matrix, clicked_vector).mean(axis=1)

    # ✅ preference와 cat2 코드가 일치하는 것에 가중치 부여
    category_data["score"] = similarities
    if preference_cat2:
        category_data.loc[category_data["cat2"] == preference_cat2, "score"] += 0.2  # 가중치 추가

    # ✅ 가장 유사한 cat3 찾기
    best_cat3 = category_data.sort_values("score", ascending=False).head(3)

    return best_cat3[["cat3", "대분류", "중분류"]].to_dict(orient="records")


In [10]:
# ✅ 추천 API 엔드포인트
@app.get("/recommend/{member_id}")
async def get_recommendations(member_id: str):
    recommendations = recommend_best_cat3(member_id)
    if not recommendations:
        raise HTTPException(status_code=404, detail="No recommendations found")
    return recommendations


In [11]:
# ✅ FastAPI 실행
if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="127.0.0.1", port=5000)

RuntimeError: asyncio.run() cannot be called from a running event loop